# Лабораторная работа №2. Курс: прикладная математика. Авторы: Ярослав Ведерников и Никита Хауров. Группа: М33011.

## Решение матричных игр

## Задачи

- Реализуйте возможность ввода данных из файла в формате JSON, который содержит матрицу игры
- Упростите платежную матрицу путем анализа доминирующих стратегий
- Если это возможно найдите решение игры в чистых стратегиях. Определите оптимальные стратегии и соответствующую цену игры
- Если решение в чистых стратегиях найти невозможно, примените симплексметод для поиска седловой точки в смешанных стратегиях. Определите смешанные стратегии и соответствующую цену игры

In [71]:
import json
import copy

# Ход работы

### Вспомогательный класс, выполняющий задачу парсинга JSON

In [72]:
class JsonTask:
    def __init__(self, json_file):
        json_content = json.loads(json_file)
        self.function = json_content["f"]
        self.goal = json_content["goal"]
        self.constraints = [phrase for phrase in json_content["constraints"]]

# Симплекс-метод

In [73]:
class Simplex:
    def __init__(self, task: JsonTask):
        if task.goal == 'min': self.function = task.function
        else: self.function = [-num for num in task.function]
        self.goal = task.goal

        self.matrix = []  # матрица симплекс метода
        for limit in task.constraints:
            if limit["type"] == 'eq':  # уравнение переводится в систему из двух нестрогих неравенств
                self.matrix.append(copy.deepcopy(limit["coefs"]) + [limit["b"]])
                self.matrix.append([-c for c in copy.deepcopy(limit["coefs"])] + [-limit["b"]])
            elif limit["type"] == 'lte': self.matrix.append(limit["coefs"] + [limit["b"]])
            elif limit["type"] == 'gte': self.matrix.append([-num for num in limit["coefs"]] + [-limit["b"]])
            else: print('Ошибка: неизвестный знак в неравенстве.')
        self.matrix.append([-num for num in self.function] + [0])  # добавляется последняя строка с целевой функцией
        self.variables_in_function = list(range(1, len(self.function) + 1))  # индексы переменных, входящих в функцию
        self.variables_in_basis = list(range(len(self.function) + 1, len(self.function) + len(self.matrix)))  # индексы переменных, входящих в базис


    def jordan_steps(self, crucial_row, crucial_column):
        matrix_before = copy.deepcopy(self.matrix)  # сохраняем матрицу до шага Жордана
        crucial_element = matrix_before[crucial_row][crucial_column]
        self.jordan_step_1(crucial_row, crucial_column)
        self.jordan_step_2(crucial_element, crucial_row, crucial_column, matrix_before)
        self.jordan_step_3(crucial_element, crucial_row, crucial_column, matrix_before)
        self.jordan_step_4(crucial_element, crucial_row, crucial_column, matrix_before)
        # чтобы иметь возможность полностью восстановить всё решение, отслеживаются индексы переменных в базисе и в функции
        excluded_var_from_basis = self.variables_in_basis.pop(crucial_row)
        excluded_var_from_func = self.variables_in_function.pop(crucial_column)
        self.variables_in_basis.insert(crucial_row, excluded_var_from_func)
        self.variables_in_function.insert(crucial_column, excluded_var_from_basis)


    def jordan_step_1(self, crucial_row, crucial_column):  # считается разрешающий элемент
        self.matrix[crucial_row][crucial_column] = 1 / self.matrix[crucial_row][crucial_column]


    def jordan_step_2(self, crucial_element, crucial_row, crucial_column, matrix_before):  # считаются элементы разрешающего столбца
        for row in (r for r in range(len(self.matrix)) if r != crucial_row):
            self.matrix[row][crucial_column] = -matrix_before[row][crucial_column] / crucial_element


    def jordan_step_3(self, crucial_element, crucial_row, crucial_column, matrix_before):  # считаются элементы разрешающей строки
        for col in (c for c in range(len(self.matrix[crucial_row])) if c != crucial_column):
            self.matrix[crucial_row][col] = matrix_before[crucial_row][col] / crucial_element


    def jordan_step_4(self, crucial_element, crucial_row, crucial_column, matrix_before):  # считаются все остальные элементы
        for row in (r for r in range(len(self.matrix)) if r != crucial_row):
            for col in (c for c in range(len(self.matrix[row])) if c != crucial_column):
                self.matrix[row][col] = matrix_before[row][col] - (matrix_before[crucial_row][col] * matrix_before[row][crucial_column] / crucial_element)


    def solve_task(self):
        self.base_plan()  # поиск опорного плана
        self.best_plan()  # поиск оптимального плана
        return self.get_correct_answer()
    

    def get_correct_answer(self):
        if (self.goal == "min"):
            return (self.matrix[-1][-1], self.variables_in_function, self.variables_in_basis, [-i for i in self.matrix[-1][:-1]], [row[-1] for row in self.matrix[:-1]])
        return (-self.matrix[-1][-1], self.variables_in_function, self.variables_in_basis, self.matrix[-1][:-1], [row[-1] for row in self.matrix[:-1]])


    def base_plan(self):
        negative_b = self.find_negative_b()
        while negative_b is not None:  # пока есть отрицательные коэффициенты в столбце свободных членов, делаем Жорданов шаг
            crucial_row, crucial_column = self.find_crucial_indices(negative_b)
            self.jordan_steps(crucial_row, crucial_column)
            negative_b = self.find_negative_b()


    def find_negative_b(self):
        for row in range(len(self.matrix) - 1):
            if self.matrix[row][-1] < 0: return row
        return None


    def find_crucial_indices(self, index_of_negative):  # ищутся индексы разрешающего элемента
        crucial_column = self.find_crucial_col(index_of_negative)
        if crucial_column is None: raise Exception('Ошибка: система ограничений несовместна.')
        return (self.find_crucial_row(crucial_column), crucial_column)


    def find_crucial_col(self, index_of_negative):
        for col in range(len(self.matrix[index_of_negative]) - 1):
            if self.matrix[index_of_negative][col] < 0: return col
        return None


    def find_crucial_row(self, crucial_column):
        min_value = float('inf')
        for row in range(len(self.matrix) - 1):
            try: val = self.matrix[row][-1] / self.matrix[row][crucial_column]
            except ZeroDivisionError: continue

            if val > 0 and val < min_value:
                min_value = val
                crucial_row = row
        return crucial_row


    def best_plan(self):
        crucial_column = self.find_best_plan_crucial_column()
        while crucial_column is not None:  # пока есть неотрицательные коэффициенты в строке целевой функции, делаем Жорданов шаг
            crucial_row = self.find_best_plan_crucial_row(crucial_column)
            if (crucial_row is None): raise Exception('Ошибка: функция не ограничена.')
            self.jordan_steps(crucial_row, crucial_column)
            crucial_column = self.find_best_plan_crucial_column()


    def find_best_plan_crucial_column(self):
        for col in range(len(self.matrix[-1]) - 1):
            if self.matrix[-1][col] > 0: return col
        return None


    def find_best_plan_crucial_row(self, crucial_column):
        min_value = float('inf')
        for row in range(len(self.matrix) - 1):
            try: val = self.matrix[row][-1] / self.matrix[row][crucial_column]
            except ZeroDivisionError: continue

            if self.matrix[row][crucial_column] > 0 and val < min_value:
                min_value = val
                crucial_row = row
        if min_value != float('inf'): return crucial_row
        return None

# Класс игры

In [74]:
class MatrixPlay:    
    def __init__(self, json_matrix):
        matrix = json.loads(json_matrix)
        self.matrix = [row["row"] for row in matrix['coefficients']]
        self.added_value = 0

    
    def is_strategy_dominated(target, dominant, player_1: bool):
        if player_1:
            for i in range(len(target)):
                if target[i] > dominant[i]: return False
        else:
            for i in range(len(target)):
                if target[i] < dominant[i]: return False
        
        return True
    

    def get_probabilities(func_vars, b_column, num_of_vars_to_find):
        vars_to_find = [i + 1 for i in range(num_of_vars_to_find)]
        probabilities = []
        for target in vars_to_find:
            try:
                fv = func_vars.index(target)
                probabilities.append(b_column[fv])
            except ValueError:
                probabilities.append(0)
        
        return MatrixPlay.normalize(probabilities)
    

    def normalize(probs):
        probs_sum = sum(probs)
        for i in range(len(probs)): probs[i] /= probs_sum
        return probs
            

    def remove_bad_strategies(self):
        have_rows_to_remove, have_columns_to_remove = True, True
        while have_columns_to_remove or have_rows_to_remove:
            have_rows_to_remove = self.remove_bad_rows(self.find_bad_rows())
            have_columns_to_remove = self.remove_bad_columns(self.find_bad_columns())

    
    def remove_bad_rows(self, rows_to_remove):
        if not rows_to_remove: return False

        for r in reversed(rows_to_remove): self.matrix.pop(r)
        return True

    
    def remove_bad_columns(self, columns_to_remove):
        if not columns_to_remove: return False

        for c in reversed(columns_to_remove):
            for i in range(len(self.matrix)): self.matrix[i].pop(c)
        return True
        
    
    def _find_solution_clear_strategies(self):
        row_1, column_1 = self.clear_strategy_1()
        row_2, column_2 = self.clear_strategy_2()
        if self.matrix[row_1][column_1] == self.matrix[row_2][column_2]: return (self.matrix[row_1][column_1], row_1, column_1)
        else: raise Exception("No solution in clear strategies.")
    

    def mix_strategies_solution(self):
        self.make_positives()
        min_task = self.compose_minimizing_task()
        max_task = self.compose_maximizing_task()
        sm_min = Simplex(min_task)
        sm_max = Simplex(max_task)

        min_value, min_func_vars, min_basis_vars, _, min_b_column = sm_min.solve_task()
        _, max_func_vars, max_basis_vars, _, max_b_column = sm_max.solve_task()
        probabilities_player_1 = MatrixPlay.get_probabilities(min_basis_vars, min_b_column, len(min_func_vars))
        probabilities_player_2 = MatrixPlay.get_probabilities(max_basis_vars, max_b_column, len(max_func_vars))
        game_price = 1 / min_value + (self.added_value)
        return (game_price, probabilities_player_1, probabilities_player_2)
    

    def compose_minimizing_task(self):
        func_coefs = [1 for x in self.matrix]
        goal = "min"
        constraints = [{"coefs": [row[c] for row in self.matrix],
                        "type": "gte",
                        "b": 1}
                        for c in range(len(self.matrix[0]))]
        
        return JsonTask(json.dumps({"f": func_coefs, "goal": goal, "constraints": constraints}))


    def compose_maximizing_task(self):
        func_coefs = [1 for y in self.matrix[0]]
        goal = "max"
        constraints = [{"coefs": [col_val for col_val in row],
                        "type": "lte",
                        "b": 1}
                        for row in self.matrix]
        
        return JsonTask(json.dumps({"f": func_coefs, "goal": goal, "constraints": constraints}))


    def make_positives(self):
        self.added_value = min([min(row) for row in self.matrix]) - 1
        if self.added_value <= 0:
            for r in range(len(self.matrix)):
                for c in range(len(self.matrix[r])): self.matrix[r][c] += (-self.added_value)
        else: self.added_value = 0


    def clear_strategy_1(self):  # finds maximum value among each row minimum values
        max_minimum_value, max_row, max_col = float("-inf"), -1, -1

        for row in range(len(self.matrix)):
            row_min_value, min_row, min_col = float("inf"), -1, -1

            for column in range(len(self.matrix[row])):
                if self.matrix[row][column] < row_min_value:
                    row_min_value, min_row, min_col = self.matrix[row][column], row, column

            if row_min_value > max_minimum_value:
                max_minimum_value, max_row, max_col = row_min_value, min_row, min_col

        return (max_row, max_col)
        

    def clear_strategy_2(self):  # finds minimum value among each column maximum values
        min_maximum_value, min_row, min_col = float("inf"), -1, -1

        for column in range(len(self.matrix[0])):
            column_max_value, max_row, max_col = float("-inf"), -1, -1

            for row in range(len(self.matrix)):
                if self.matrix[row][column] > column_max_value:
                    column_max_value, max_row, max_col = self.matrix[row][column], row, column

            if column_max_value < min_maximum_value:
                min_maximum_value, min_row, min_col = column_max_value, max_row, max_col
                
        return (min_row, min_col)


    def find_bad_rows(self):
        dominated_rows_indices = []
        for row_target in range(len(self.matrix)):
            for row_dominant in range(len(self.matrix)):
                if row_target != row_dominant and MatrixPlay.is_strategy_dominated(self.matrix[row_target], self.matrix[row_dominant], True):
                    dominated_rows_indices.append(row_target)
                    break

        return dominated_rows_indices


    def find_bad_columns(self):
        dominated_columns_indices = []
        for column_target in range(len(self.matrix[0])):
            target_values = [row[column_target] for row in self.matrix]
            for column_dominant in range(len(self.matrix[0])):
                dominant_values = [row[column_dominant] for row in self.matrix]
                if column_target != column_dominant and MatrixPlay.is_strategy_dominated(target_values, dominant_values, False):
                    dominated_columns_indices.append(column_target)
                    break

        return dominated_columns_indices
    

    def find_solution(self):
        self.remove_bad_strategies()
        print(f"Матрица игры после удаления доминирующих стратегий:\n {self.matrix}")
        try:
            clear_solution = self._find_solution_clear_strategies()
            print(f"Лучшая стратегии для игрока 1: {clear_solution[1]}\n"
                  f"Лучшая стратегии для игрока 2: {clear_solution[2]}\n"
                  f"Цена игры: {clear_solution[0]}")
            return clear_solution
        except Exception as ex:
            mixed_solution = self.mix_strategies_solution()
            print(f"Вероятности стратегий для игрока 1: {mixed_solution[1]}\n"
                  f"Вероятности стратегий для игрока 2: {mixed_solution[2]}\n"
                  f"Цена игры: {mixed_solution[0]}")
            return mixed_solution

# Примеры игр с решениями

In [75]:
test_1 = '{"coefficients": [{"row": [1, 2, 3, -1]},{"row": [2, -2, 2, -2]},{"row": [3, 0, 3, 1]}]}'
_ = MatrixPlay(test_1).find_solution()

Матрица игры после удаления доминирующих стратегий:
 [[2, -1], [0, 1]]
Вероятности стратегий для игрока 1: [0.24999999999999997, 0.75]
Вероятности стратегий для игрока 2: [0.5, 0.5]
Цена игры: 0.5


In [76]:
test_2 = '{"coefficients": [{"row": [-23, -7, -2]},{"row": [0, -2, -6]},{"row": [-2, -2, -3]}]}'
_ = MatrixPlay(test_2).find_solution()

Матрица игры после удаления доминирующих стратегий:
 [[-23, -7, -2], [0, -2, -6], [-2, -2, -3]]
Вероятности стратегий для игрока 1: [0.045454545454546594, 0.0, 0.9545454545454535]
Вероятности стратегий для игрока 2: [0.04545454545454539, 0.0, 0.9545454545454546]
Цена игры: -2.9545454545454923


In [77]:
test_3 = '{"coefficients": [{"row": [-1, 3]},{"row": [3, -5]}]}'
_ = MatrixPlay(test_3).find_solution()

Матрица игры после удаления доминирующих стратегий:
 [[-1, 3], [3, -5]]
Вероятности стратегий для игрока 1: [0.6666666666666666, 0.3333333333333333]
Вероятности стратегий для игрока 2: [0.6666666666666666, 0.3333333333333333]
Цена игры: 0.3333333333333339


In [78]:
test_4 = '{"coefficients": [{"row": [5, 10]},{"row": [1, 3]}]}'
_ = MatrixPlay(test_4).find_solution()

Матрица игры после удаления доминирующих стратегий:
 [[5]]
Лучшая стратегии для игрока 1: 0
Лучшая стратегии для игрока 2: 0
Цена игры: 5


## Вывод:

По итогам лаборатоной работы удалось реализовать решение матричных игр с использованием симплекс-метода. Также реализовано удаление доминирующих стратегий и поиск решения в чистых стратегиях.